[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hammad-ali1/vad_clip_colab_notebook/blob/main/vadclip_feature_extraction.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
EXTRACTED_DATASET =  '/content/drive/MyDrive/Training-Normal-Videos-Part-2'
SAVE_DIR = '/content/drive/MyDrive/UCFMambaClipFeatures'

In [3]:
import os
import glob

assert os.path.exists(SAVE_DIR)

processed_files = glob.glob(os.path.join(SAVE_DIR, "**", "*.npy"), recursive=True)
print(f'Already processed files: {len(processed_files) // 10}')

Already processed files: 1530


In [4]:
assert os.path.exists(EXTRACTED_DATASET)

video_files = glob.glob(os.path.join(EXTRACTED_DATASET, "**", "*.mp4"), recursive=True)
print(f'Total Videos: {len(video_files)}')

Total Videos: 370


In [5]:
!pip -q install ftfy
!pip -q install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0
!pip -q install causal-conv1d==1.4.0 && pip install mamba-ssm==2.2.2
!pip -q install fvcore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0

In [6]:
#@title Pull Latest Repo
from google.colab import userdata
from pathlib import Path

try:
  gh_token = userdata.get('vad_clip_gh_token')
except Exception:
  gh_token = None

REPO_NAME = "vad_clip_colab_notebook"
REPO_PATH = Path("/content") / REPO_NAME

if gh_token:
  GITHUB_URL = f"https://{gh_token}@github.com/hammad-ali1/{REPO_NAME}.git"
else:
  GITHUB_URL = f'https://github.com/hammad-ali1/{REPO_NAME}'

!rm -rf {REPO_PATH}
!git clone {GITHUB_URL}

!cp -r {REPO_PATH}/* /content

Cloning into 'vad_clip_colab_notebook'...
remote: Enumerating objects: 256, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 256 (delta 137), reused 161 (delta 78), pack-reused 0 (from 0)
Receiving objects: 100% (256/256), 3.43 MiB | 3.47 MiB/s, done.
Resolving deltas: 100% (137/137), done.


In [7]:
!cd /content/vmamba/kernels/selective_scan && pip install .

Processing /content/vmamba/kernels/selective_scan
  Preparing metadata (setup.py) ... done
  Created wheel for selective_scan: filename=selective_scan-0.0.2-cp312-cp312-linux_x86_64.whl size=9293919 sha256=a1f145094079387087523a86b5d3e1bf523cdd20b5fe6eceb9563842cb8fb9c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-thox1khc/wheels/e0/f2/5a/bfc38021b58688bf7ba0131e4895249de8417294087d81d6bb
Successfully built selective_scan


In [8]:
import torch
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize

import itertools
from more_itertools import chunked
import gc
from collections import OrderedDict

from models import CLIP_VMamba_S

import cv2
import numpy as np
from PIL import Image
from tqdm.auto import tqdm

/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')`

In [9]:
def video_crop(video_frame, type):
    l = video_frame.shape[0]
    new_frame = []
    for i in range(l):
        img = cv2.resize(video_frame[i], dsize=(340, 256))
        new_frame.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    #1
    img = np.array(new_frame)
    del new_frame
    if type == 0:
        img = img[:, 16:240, 58:282, :]
    #2
    elif type == 1:
        img = img[:, :224, :224, :]
    #3
    elif type == 2:
        img = img[:, :224, -224:, :]
    #4
    elif type == 3:
        img = img[:, -224:, :224, :]
    #5
    elif type == 4:
        img = img[:, -224:, -224:, :]
    #6
    elif type == 5:
        img = img[:, 16:240, 58:282, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)
    #7
    elif type == 6:
        img = img[:, :224, :224, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)
    #8
    elif type == 7:
        img = img[:, :224, -224:, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)
    #9
    elif type == 8:
        img = img[:, -224:, :224, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)
    #10
    elif type == 9:
        img = img[:, -224:, -224:, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)

    return img

def image_crop(image, type):
    img = cv2.resize(image, dsize=(340, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #1
    if type == 0:
        img = img[16:240, 58:282, :]
    #2
    elif type == 1:
        img = img[:224, :224, :]
    #3
    elif type == 2:
        img = img[:224, -224:, :]
    #4
    elif type == 3:
        img = img[-224:, :224, :]
    #5
    elif type == 4:
        img = img[-224:, -224:, :]
    #6
    elif type == 5:
        img = img[16:240, 58:282, :]
        img = cv2.flip(img, 1)
    #7
    elif type == 6:
        img = img[:224, :224, :]
        img = cv2.flip(img, 1)
    #8
    elif type == 7:
        img = img[:224, -224:, :]
        img = cv2.flip(img, 1)
    #9
    elif type == 8:
        img = img[-224:, :224, :]
        img = cv2.flip(img, 1)
    #10
    elif type == 9:
        img = img[-224:, -224:, :]
        img = cv2.flip(img, 1)

    return img

In [10]:
def video_frame_generator(video_path, stride=16, pick='center'):
    assert stride >= 1
    cap = cv2.VideoCapture(video_path)
    chunk = []
    center_idx = (stride // 2 - 1) if (stride % 2 == 0) else (stride // 2)

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        chunk.append(frame)

        if len(chunk) == stride:
            if pick == 'center':
                idx = center_idx
            elif pick == 'first':
                idx = 0
            elif pick == 'last':
                idx = stride - 1
            else:
                raise ValueError(f"Unknown pick='{pick}'")

            yield chunk[idx]
            chunk.clear()

    cap.release()

In [11]:
def prepare_feature_path(video_path, save_dir):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    parent_dir = os.path.basename(os.path.dirname(video_path))
    out_parent = os.path.join(save_dir, parent_dir)
    os.makedirs(out_parent, exist_ok=True)

    return video_name, out_parent


def crops_exist(out_parent, video_name):
    return all(os.path.exists(os.path.join(out_parent, f"{video_name}__{c}.npy")) for c in range(10))


def get_frame_count(video_path):
    """Efficiently gets the total number of frames in a video."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return 0
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count


def save_clip_features(video_path, save_dir, model, preprocess, device,
                          stride=16, pick='center', frames_chunk_size=2 ** 14, batch_size=256, crop_range=(0, 10)):

    video_name, out_parent = prepare_feature_path(video_path, save_dir)
    crops_to_process = [c for c in range(*crop_range) if
                          not os.path.exists(os.path.join(out_parent, f"{video_name}__{c}.npy"))]

    if not crops_to_process:
        return

    features_per_crop = [[] for _ in range(max(crops_to_process) + 1)]
    total_sampled_frames = get_frame_count(video_path) // stride
    desc=f"Processing {video_name}. Crops: [{min(crops_to_process)} - {max(crops_to_process)}]"

    with tqdm(total=total_sampled_frames, desc=desc, unit="frame", position=1, leave=False) as pbar:
      for frames_chunk in chunked(video_frame_generator(video_path, stride, pick),
                                frames_chunk_size):

        frames_chunk = np.array(list(frames_chunk))

        if frames_chunk.size == 0:
          raise ValueError(f"No frames found (after sampling) in {video_path}")

        for batch_frames in chunked(frames_chunk, batch_size):
            batch_frames = np.array(list(batch_frames))

            for crop_type in crops_to_process:
                cropped_frames = video_crop(batch_frames, crop_type)  # expects (N, H, W, C)

                batch_tensors = [preprocess(Image.fromarray(f)) for f in cropped_frames]
                batch_tensors = torch.stack(batch_tensors).to(device)

                with torch.no_grad():
                  feats = model.encode_image(batch_tensors)

                features_per_crop[crop_type].append(feats.cpu().numpy())

            pbar.update(len(batch_frames))

    for crop_type in crops_to_process:
        save_path = os.path.join(out_parent, f"{video_name}__{crop_type}.npy")

        if features_per_crop[crop_type]:
            video_features = np.vstack(features_per_crop[crop_type])
            np.save(save_path, video_features)


In [12]:
def extract_dataset_features(dataset_dir, save_dir, model, preprocess, device):
    video_files = glob.glob(os.path.join(dataset_dir, "**", "*.mp4"), recursive=True)
    video_files.sort()

    for video_path in tqdm(video_files, desc="Dataset", unit="video", position=0):
        try:
            video_size = os.path.getsize(video_path) / (1024 ** 3)
            if video_size > 4:
              for crop_range in [(i, i+1) for i in range(10)]:
                save_clip_features(
                        video_path=video_path,
                        save_dir=save_dir,
                        model=model,
                        preprocess=preprocess,
                        device=device,
                        crop_range=crop_range
                    )
            else:
              save_clip_features(
                        video_path=video_path,
                        save_dir=save_dir,
                        model=model,
                        preprocess=preprocess,
                        device=device
                    )
        except Exception as e:
            print(f"⚠️ Error processing {video_path}: {e}")


In [13]:
def load_clip_model():
  !curl -L -o VMamba_S_clip.pt "https://huggingface.co/weiquan/mamba-clip/resolve/main/VMamba_S_clip.pt?download=true"

  mamba_clip_model = CLIP_VMamba_S()

  ckpt_path = "VMamba_S_clip.pt"

  ckpt = torch.load(ckpt_path, map_location='cpu', weights_only=False)
  state_dict = OrderedDict()
  for k, v in ckpt['state_dict'].items():
      state_dict[k.replace('module.', '')] = v

  mamba_clip_model.load_state_dict(state_dict, strict=True)

  return mamba_clip_model

In [14]:
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC


def _convert_image_to_rgb(image):
    return image.convert("RGB")

def _transform(n_px=224):
    return Compose([
        Resize(n_px, interpolation=BICUBIC),
        CenterCrop(n_px),
        _convert_image_to_rgb,
        ToTensor(),
        Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ])

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = load_clip_model()
model = model.to(device)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1325  100  1325    0     0   4729      0 --:--:-- --:--:-- --:--:--  4732
100 1728M  100 1728M    0     0   185M      0  0:00:09  0:00:09 --:--:--  173M


/content/vmamba/vmamba.py:262: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/content/vmamba/vmamba.py:270: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/content/vmamba/vmamba.py:285: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/content/vmamba/vmamba.py:293: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/content/vmamba/vmamba.py:306: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/cont

=> merge config from /content/vmamba/configs/vssm1/vssm_small_224.yaml


In [ ]:
preprocess = _transform()
extract_dataset_features(EXTRACTED_DATASET, SAVE_DIR, model, preprocess, device)

Dataset:   0%|          | 0/370 [00:00<?, ?video/s]

Processing Normal_Videos483_x264. Crops: [0 - 9]:   0%|          | 0/266 [00:00<?, ?frame/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Processing Normal_Videos484_x264. Crops: [0 - 9]:   0%|          | 0/2248 [00:00<?, ?frame/s]

Processing Normal_Videos485_x264. Crops: [0 - 9]:   0%|          | 0/71 [00:00<?, ?frame/s]

...

Processing Normal_Videos507_x264. Crops: [0 - 9]:   0%|          | 0/128 [00:00<?, ?frame/s]

Processing Normal_Videos508_x264. Crops: [0 - 9]:   0%|          | 0/88 [00:00<?, ?frame/s]

Processing Normal_Videos509_x264. Crops: [0 - 9]:   0%|          | 0/43 [00:00<?, ?frame/s]

Processing Normal_Videos510_x264. Crops: [0 - 9]:   0%|          | 0/1689 [00:00<?, ?frame/s]

Processing Normal_Videos511_x264. Crops: [0 - 9]:   0%|          | 0/1688 [00:00<?, ?frame/s]